## THE CIRCUIT CONFIGURATION

In [1]:
import subprocess
import sys
#subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
#subprocess.check_call([sys.executable, "-m", "pip", "install", "solc-select"])


import json
import ezkl
import onnx
!pip list | grep ezkl
!pip list | grep solc-select
import pandas as pd

import torch
from torchvision import transforms
from PIL import Image

import logging
import os
import sys
import time
from datetime import datetime


# uncomment for more descriptive logging
FORMAT = '%(levelname)s %(name)s %(asctime)-15s %(filename)s:%(lineno)d %(message)s'
logging.basicConfig(format=FORMAT)
logging.getLogger().setLevel(logging.INFO)

ModuleNotFoundError: No module named 'ezkl'

### DNN circuit calibration

In [23]:
model_path = os.path.join('outputs/dnn/model460.onnx')
settings_path = os.path.join('outputs/dnn/settings.json')
cal_path = os.path.join('outputs/dnn/calib_data460.json')

#For ezkl to compute a snark, it needs some settings to determine how to create the circuit.
#This cell instantiates some parameters that determine the circuit shape, size etc
py_run_args = ezkl.PyRunArgs()
py_run_args.input_visibility = "private"
py_run_args.output_visibility = "public"
py_run_args.param_visibility = "fixed" # "fixed" for params means that the committed to params are used for all proofs
py_run_args.variables = [("batch_size", 1)]

os.environ["RUST_BACKTRACE"] = "1"

!RUST_LOG=trace
# TODO: Dictionary outputs
start_time = time.time()
res = ezkl.gen_settings(model_path, settings_path, py_run_args=py_run_args)
end_time = time.time()
assert res == True
print("gen_settings done in: ", end_time - start_time, "seconds")

os.environ["RUST_BACKTRACE"] = "1"

# *** SCALES is one of main knobs you can turn to trade off accuracy for proving efficiency
# Under the hood calibration iterates over the scales array to see how precise you can go before failure.
# For example if 7 fails it falls back to 1.
# (Right now the default scales values for the resources target is 8-10 and 10-13  for accuracy)
start_time = time.time()
#res = await ezkl.calibrate_settings(cal_path, model_path, settings_path, "resources", scales=[3], scale_rebase_multiplier=[0,4])
res = await ezkl.calibrate_settings(cal_path, model_path, settings_path, "accuracy", max_logrows=21, lookup_safety_margin=2, scales=[8])#, scale_rebase_multiplier=[1])
end_time = time.time()
print("calibrate_settings done in", end_time - start_time, "seconds")

INFO ezkl.execute 2024-07-12 16:16:25,275 execute.rs:1037 num calibration batches: 20


gen_settings done in:  0.7814228534698486 seconds


WARNING ezkl.execute 2024-07-12 16:18:23,109 execute.rs:1356 

 <------------- Numerical Fidelity Report (input_scale: 8, param_scale: 8, scale_input_multiplier: 10) ------------->

+-------------+--------------+-------------+--------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error  | median_error | max_error   | min_error    | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+-------------+--------------+-------------+--------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| 0.008295262 | 0.018511027  | 0.018511027 | -0.004630387 | 0.008758301    | 0.018511027      | 0.018511027   | 0.0020294487  | 0.00010289586      | -0.05094374        | 0.05440534             |
+-------------+--------------+--------

calibrate_settings done in 117.93903017044067 seconds


In [13]:


# Visualize the circuit settings
settings_path = os.path.join('outputs/dnn/settings.json')
json_file_path = settings_path
# Read the content of the JSON file
with open(json_file_path, 'r') as file:
    json_content = json.load(file)

# Print the JSON content
print(json.dumps(json_content, indent=2))

{
  "run_args": {
    "tolerance": {
      "val": 0.0,
      "scale": 1.0
    },
    "input_scale": 3,
    "param_scale": 3,
    "scale_rebase_multiplier": 4,
    "lookup_range": [
      -154,
      186
    ],
    "logrows": 17,
    "num_inner_cols": 2,
    "variables": [
      [
        "batch_size",
        1
      ]
    ],
    "input_visibility": "Private",
    "output_visibility": "Public",
    "param_visibility": "Fixed",
    "div_rebasing": false,
    "rebase_frac_zero_constants": false,
    "check_mode": "UNSAFE",
    "commitment": "KZG"
  },
  "num_rows": 116014,
  "total_assignments": 232028,
  "total_const_size": 4327,
  "total_dynamic_col_size": 0,
  "num_dynamic_lookups": 0,
  "num_shuffles": 4320,
  "total_shuffle_col_size": 17280,
  "model_instance_shapes": [
    [
      1,
      1
    ]
  ],
  "model_output_scales": [
    9
  ],
  "model_input_scales": [
    3,
    3
  ],
  "module_sizes": {
    "polycommit": [],
    "poseidon": [
      0,
      [
        0
      ]
    ]

In [13]:
res = await ezkl.get_srs(settings_path=None, logrows=19, commitment=ezkl.PyCommitments.KZG)

INFO ezkl.execute 2024-07-10 15:34:27,883 execute.rs:716 SRS does not exist, downloading...
INFO ezkl.execute 2024-07-10 15:38:09,134 execute.rs:728 Saving SRS to disk...
INFO ezkl.execute 2024-07-10 15:38:09,190 execute.rs:733 Saved SRS to disk.
INFO ezkl.execute 2024-07-10 15:38:09,190 execute.rs:735 SRS downloaded
INFO ezkl.execute 2024-07-10 15:38:09,201 execute.rs:639 read 67109124 bytes from file (vector of len = 67109124)
INFO ezkl.execute 2024-07-10 15:38:09,386 execute.rs:646 file hash: d1a1655b4366a766d1578beb257849a92bf91cb1358c1a2c37ab180c5d3a204d
INFO ezkl.execute 2024-07-10 15:39:26,808 execute.rs:728 Saving SRS to disk...
INFO ezkl.execute 2024-07-10 15:39:26,846 execute.rs:733 Saved SRS to disk.
INFO ezkl.execute 2024-07-10 15:39:26,847 execute.rs:735 SRS downloaded
INFO ezkl.execute 2024-07-10 15:39:26,857 execute.rs:639 read 67109124 bytes from file (vector of len = 67109124)
INFO ezkl.execute 2024-07-10 15:39:27,049 execute.rs:646 file hash: d1a1655b4366a766d1578beb2

### DNN Setup

In [15]:
# Compile the model into a circuit
compiled_model_path = os.path.join('outputs/dnn/dnn460.ezkl')
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

# Get public srs from kzg ceremony.
settings_path = os.path.join('outputs/dnn/settings.json')
res = ezkl.get_srs(settings_path)

INFO ezkl.execute 2024-07-12 16:00:47,059 execute.rs:741 SRS already exists at that path


INFO ezkl.execute 2024-07-12 16:00:47,062 execute.rs:639 read 16777476 bytes from file (vector of len = 16777476)
INFO ezkl.execute 2024-07-12 16:00:47,163 execute.rs:646 file hash: 41509f380362a8d14401c5ae92073154922fe23e45459ce6f696f58607655db7


In [6]:
# Generate a larger SRS. This is needed for the aggregated proof

res = await ezkl.get_srs(settings_path=None, logrows=20, commitment=ezkl.PyCommitments.KZG)

INFO ezkl.execute 2024-07-11 11:04:04,974 execute.rs:716 SRS does not exist, downloading...
INFO ezkl.execute 2024-07-11 11:06:42,587 execute.rs:728 Saving SRS to disk...
INFO ezkl.execute 2024-07-11 11:06:42,682 execute.rs:733 Saved SRS to disk.
INFO ezkl.execute 2024-07-11 11:06:42,683 execute.rs:735 SRS downloaded
INFO ezkl.execute 2024-07-11 11:06:42,702 execute.rs:639 read 134217988 bytes from file (vector of len = 134217988)
INFO ezkl.execute 2024-07-11 11:06:43,077 execute.rs:646 file hash: 54ef75911da76d7a6b7ea341998aaf66cb06c679c53e0a88a4fe070dd3add963
INFO ezkl.execute 2024-07-11 11:06:54,887 execute.rs:728 Saving SRS to disk...
INFO ezkl.execute 2024-07-11 11:06:54,945 execute.rs:733 Saved SRS to disk.
INFO ezkl.execute 2024-07-11 11:06:54,946 execute.rs:735 SRS downloaded
INFO ezkl.execute 2024-07-11 11:06:54,964 execute.rs:639 read 134217988 bytes from file (vector of len = 134217988)
INFO ezkl.execute 2024-07-11 11:06:55,328 execute.rs:646 file hash: 54ef75911da76d7a6b7ea

In [17]:
# Setup the circuit and make sure the keys are generated afterwards.
pk_path = os.path.join('outputs/dnn/pk460.key')
vk_path = os.path.join('outputs/dnn/vk460.key')

import os
import time
import psutil
import json

process = psutil.Process(os.getpid())  # Get the current process
start_time = time.time()
start_cpu_times = process.cpu_times()  # CPU times at the start
start_memory = process.memory_info().rss  # Memory usage at the start

os.environ["RUST_BACKTRACE"] = "1"

res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
    )

end_time = time.time()

execution_time = end_time - start_time

vk_size = os.path.getsize(vk_path)
pk_size = os.path.getsize(pk_path)

end_time = time.time()
end_cpu_times = process.cpu_times()  # CPU times at the end
end_memory = process.memory_info().rss  # Memory usage at the end

execution_time = end_time - start_time
cpu_time_user = end_cpu_times.user - start_cpu_times.user
cpu_time_system = end_cpu_times.system - start_cpu_times.system
total_cpu_time = cpu_time_user + cpu_time_system
memory_used = (end_memory - start_memory) / (1024 * 1024)  # Convert to MB

print(f"Setup took {execution_time:.2f} seconds (real-time).")
print(f"CPU time used: User - {cpu_time_user:.2f} seconds, System - {cpu_time_system:.2f} seconds.")
print(f"Total CPU time: {total_cpu_time:.2f} seconds.")
print(f"Memory used: {memory_used:.2f} MB")


assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)

INFO ezkl.graph.model 2024-07-12 16:01:52,926 model.rs:1083 model layout...
INFO ezkl.pfsys 2024-07-12 16:01:57,578 mod.rs:520 VK took 4.662
INFO ezkl.graph.model 2024-07-12 16:01:57,584 model.rs:1083 model layout...
INFO ezkl.pfsys 2024-07-12 16:02:02,867 mod.rs:526 PK took 5.288
INFO ezkl.pfsys 2024-07-12 16:02:02,871 mod.rs:844 done saving verification key ✅
INFO ezkl.pfsys 2024-07-12 16:02:03,373 mod.rs:827 done saving proving key ✅


Setup took 10.50 seconds (real-time).
CPU time used: User - 33.48 seconds, System - 14.43 seconds.
Total CPU time: 47.91 seconds.
Memory used: 86.33 MB


In [19]:
# Generate the verifier associated with the circuit
sol_code_path = os.path.join('outputs/dnn/Verifier460.sol')
abi_path = os.path.join('outputs/dnn/Verifier460.abi')
vk_path = os.path.join('outputs/dnn/vk460.key')
settings_path = os.path.join('outputs/dnn/settings.json')

res = ezkl.create_evm_verifier(
        vk_path,
        settings_path,
        sol_code_path,
        abi_path
    )
#assert res == True
#assert os.path.isfile(sol_code_path)

INFO ezkl.pfsys 2024-07-12 16:03:48,130 mod.rs:787 loaded verification key ✅


## DNN Prove
### here we will generate and verify a proof locally. Then we will format the inputs and the proof in a way compatible for the evm verifier

In [20]:
witness_path = os.path.join('outputs/dnn/witness460.json')
data_path = os.path.join('outputs/dnn/input460.json')

# generate the witness file
res = ezkl.gen_witness(data_path, compiled_model_path, witness_path)

In [21]:
import psutil
# Generate the proof
proof_path = os.path.join('outputs/dnn/proof460.json')
#witness_path = os.path.join('outputs/dnn/witness23k.json')
#pk_path = os.path.join('outputs/dnn/pk23k.key')
#compiled_model_path = os.path.join('outputs/dnn/dnn23k.ezkl')


process = psutil.Process(os.getpid())  # Get the current process
start_time = time.time()
start_cpu_times = process.cpu_times()  # CPU times at the start
start_memory = process.memory_info().rss  # Memory usage at the start

proof = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,
        "single",
    )

end_time = time.time()
end_cpu_times = process.cpu_times()  # CPU times at the end
end_memory = process.memory_info().rss  # Memory usage at the end

execution_time = end_time - start_time
cpu_time_user = end_cpu_times.user - start_cpu_times.user
cpu_time_system = end_cpu_times.system - start_cpu_times.system
total_cpu_time = cpu_time_user + cpu_time_system
memory_used = (end_memory - start_memory) / (1024 * 1024)  # Convert to MB

print(f"proof took {execution_time:.2f} seconds (real-time).")
print(f"CPU time used: User - {cpu_time_user:.2f} seconds, System - {cpu_time_system:.2f} seconds.")
print(f"Total CPU time: {total_cpu_time:.2f} seconds.")
print(f"Memory used: {memory_used:.2f} MB")

print(proof)

INFO ezkl.pfsys 2024-07-12 16:04:11,230 mod.rs:810 loaded proving key ✅
INFO ezkl.pfsys 2024-07-12 16:04:11,238 mod.rs:581 proof started...
INFO ezkl.graph.model 2024-07-12 16:04:11,265 model.rs:1083 model layout...
INFO ezkl.pfsys 2024-07-12 16:04:25,086 mod.rs:620 proof took 13.847


proof took 14.45 seconds (real-time).
CPU time used: User - 54.56 seconds, System - 8.32 seconds.
Total CPU time: 62.88 seconds.
Memory used: 61.27 MB
{'instances': [['c600000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x2579c8b89201083149bcc0a41549bada0d29b63eb0a99813f87db62d1311aa5d0ed3eb9c91d72dc82e55165c100ce95e9daaff3d84453de9595c3db6f74ba7892395d47039ae4b54c4d27d364248fc9bbf83b9fe22473ccfb749e29abd1c3e710c41e539a4014debcb42965bfecac700d135a32b92f25e0e224d0fc748333a201d657cc45f37fa5f942f9f9fff2fb632778b2704256aa31873d7e321194ad81f0bb1e4b1a5353f7387f24bceb9dd87712722eb833e88dd7a1bc638357c17fbca2dd7dce65aa20ef60096e4d6adf36b30f150b9e61cb7e7ccfdfbf407dd8430841f5f0b90866b879e6526c009662ebaf0214950d98f334358fd3b113ede208cae0f18d700fad294485abd366937ede13f64ac46585b1388e751a34dd33fc5ed3c178c9bd910d003371fce20272bb1c2d3040bfa8c20e4c6a1884cd2827100c247027257e9baabb4a3cb82c4534e608108fb78ec2b8afb1e9aebf2ce9e0dbb6e750dc483426c750867658b5af0bdbee7597a49eba6d14c293

In [25]:
# Sanity check off-chain verification
proof_path = os.path.join('outputs/dnn/proof460.json')
vk_path = os.path.join('outputs/dnn/vk460.key')
settings_path = os.path.join('outputs/dnn/settings.json')

process = psutil.Process(os.getpid())  # Get the current process
start_time = time.time()
start_cpu_times = process.cpu_times()  # CPU times at the start
start_memory = process.memory_info().rss  # Memory usage at the start

res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
    )

# Measure end times and resource usage
end_time = time.time()
end_cpu_times = process.cpu_times()  # CPU times at the end
end_memory = process.memory_info().rss  # Memory usage at the end

execution_time = end_time - start_time
cpu_time_user = end_cpu_times.user - start_cpu_times.user
cpu_time_system = end_cpu_times.system - start_cpu_times.system
total_cpu_time = cpu_time_user + cpu_time_system
memory_used = (end_memory - start_memory) / (1024 * 1024)  # Convert to MB

print(f"verify took {execution_time:.2f} seconds (real-time).")
print(f"CPU time used: User - {cpu_time_user:.2f} seconds, System - {cpu_time_system:.2f} seconds.")
print(f"Total CPU time: {total_cpu_time:.2f} seconds.")
print(f"Memory used: {memory_used:.2f} MB")

assert res == True
print("verified locally")

INFO ezkl.pfsys 2024-07-12 16:20:39,953 mod.rs:787 loaded verification key ✅
INFO ezkl.execute 2024-07-12 16:20:39,957 execute.rs:2453 verify took 0.3
INFO ezkl.execute 2024-07-12 16:20:39,957 execute.rs:2458 verified: false


RuntimeError: Failed to run verify: [halo2] The constraint system is not satisfied

In [4]:
onchain_input_array = []

proof_path = os.path.join('outputs/dnn/proof23k.json')
proof = json.load(open(proof_path, 'r'))

# using a loop
# avoiding printing last comma
formatted_output = "["
for i, value in enumerate(proof["instances"]):
    for j, field_element in enumerate(value):
        onchain_input_array.append(ezkl.felt_to_big_endian(field_element))
        formatted_output += '"' + str(onchain_input_array[-1]) + '"'
        if j != len(value) - 1:
            formatted_output += ", "
    if i != len(proof["instances"]) - 1:
        formatted_output += ", "
formatted_output += "]"

json_data = {
    "instances": formatted_output,
    "proof": proof["hex_proof"]
}

with open('outputs/dnn/evmInputs23k.json', 'w') as json_file:
    json.dump(json_data, json_file, indent=2)
print("Saved evmInputs.json: ", json_data)

Saved evmInputs.json:  {'instances': '["0x30644e72e131a029b85045b68181585d2833e84879b9709143e1f593efffca35"]', 'proof': '0x079f0e89d3f86dede2126c3cf6c4cec4e051d8e7e52959a56341a87a783ce95720048a84d7677d72f309ba309b4fc9a4b98a0de5d7be6f89035a4c27ae03b8f125d63f8b97b9f641e5330b575aa4c99cdea2bdffc9621a15a7f2d3b2603e829c0b2ba315b3d3e585ab34dc04772032107fcd330fc369dba0bccd2717963d73241c8e11d5b42977dcd4235efcb3c7429b8b0eec11758c4ef7221b733e2c41a5ed2bf078a4c387bbee1994c8d847c8d9f4f3c88dcd92d77f738b5b9db1659d097c0850b5c344a599fd36f68a9104caadf4ade7e32cc750e5319a2e32586dddcaef0d599cd321866ef1854c5c40296f88bd1b08419f0eb428a7a13c7d8184cfbdfe08fec5a561592c103ff2e2b441f82e7178b98c6349b0ccc86797593a931c902d1a1eedff61c8edab16ee67e84f95dbe01746051efabbd09e9828f415cd41401f0e06cd04222cc750e63c610dfc78ca7528b4337afbd5d264665e9f5e9111f707219c93839cde71e63b4e35961af2afd9a2f6a571cd6bfd2e72e3f4d7f464e2342566430ab76cc410dc7a5e7505a702be20bd87a1a3779c9198a6b40509da72a1087ca4be0a2738bb03394720898af9ee35240632867cc